In [52]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import ssl
import numpy as np

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Falls das Attribut nicht existiert, gibt es nichts zu tun
    pass
else:
    # Setzt den Standard-SSL-Kontext auf einen unbestätigten Kontext
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielzellner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/danielzellner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/danielzellner/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielzellner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [44]:
def lemmatize_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(word, wordnet.VERB) for word in tokens]
    return ' '.join(lemmatized_tokens)

In [9]:
df = pd.read_csv("merged_reviews_metadata_renamed.csv", sep=";")

display(df.head())

,review_rating,review_title,review_text,product_asin,product_parent_asin,review_user_id,helpful_votes,is_verified_purchase,product_main_category,product_title,product_average_rating,product_rating_count,product_store,products_bought_together
0,5,Most Excellent,Works great and sounds out loud!,B005QUF0C2,B005QUF0C2,AGWPAWFHA6EJJNSYYF3AXPMV4WSA,0,True,Musical Instruments,VoiceBooster Voice Amplifier 20watts Black MR2...,4.6,107,Voice Booster,NaN
1,3,Three Stars,"The knobs seem like okay quality, but they don...",B0077RI0QA,B0077RI0QA,AFPN25YIVEINVD7I6DNEC2ALFLBQ,5,True,Musical Instruments,8-pack Potentiometer Knobs: Black Ribbed with ...,3.8,19,C. B. Gitty,NaN
2,5,Very durable and produces a great sound,Very durable and produce great sound. I person...,B00BL6JDUA,B08BJGBHN7,AHTNYO2ZE6FAGDXMMTKSZAGZAXGA,0,True,Musical Instruments,ChromaCast 96 Pick Sampler - Includes Pearl Ce...,4.7,9760,ChromaCast,NaN
3,5,Perfect fit for my 30 year old Crybaby,I lost a few of these over the years and had s...,B003B0YDRM,B003B0YDRM,AGZL7HSQMHJUY3SUORJGSXKIHUDQ,1,True,AMAZON FASHION,"JIM DUNLOP ECB151 Rubber Feet with Screws, 4/Bag",4.8,503,JIM DUNLOP,NaN
4,3,Meh,One of the pick holders broke instantly. Will ...,B07YFM4X6X,B07YFM4X6X,AFH5OO43G3DPYJ75N7YQTUXKJNXA,0,True,Musical Instruments,20 Pieces Metal Guitar Picks Plectrums Stainle...,4.4,1243,Frienda,NaN


In [13]:
category_counts = df['product_main_category'].value_counts()

print(category_counts)

product_main_category
Musical Instruments             129669
All Electronics                   5751
Home Audio & Theater              2111
Industrial & Scientific           2018
Tools & Home Improvement          1912
Amazon Home                       1896
Toys & Games                       950
Unknown                            914
AMAZON FASHION                     908
Computers                          827
Cell Phones & Accessories          678
Sports & Outdoors                  353
Office Products                    337
Camera & Photo                     256
Books                              234
Automotive                         232
Software                           229
Health & Personal Care             179
Arts, Crafts & Sewing              111
Car Electronics                     84
All Beauty                          79
Pet Supplies                        68
Video Games                         22
Baby                                19
Portable Audio & Accessories        16
Gro

In [22]:
# Delete unused columns

df_clean = df[df['product_main_category'] == 'Musical Instruments'][[
    'review_rating', 'review_title', 'review_text', 'helpful_votes',
    'is_verified_purchase', 'product_title', 'product_average_rating',
    'product_rating_count', 'products_bought_together'
]]
print(df_clean.head())
print(df_clean.dtypes)

   review_rating                             review_title  \
0              5                           Most Excellent   
1              3                              Three Stars   
2              5  Very durable and produces a great sound   
4              3                                      Meh   
5              4             Displays our Pick Collection   

                                         review_text  helpful_votes  \
0                   Works great and sounds out loud!              0   
1  The knobs seem like okay quality, but they don...              5   
2  Very durable and produce great sound. I person...              0   
4  One of the pick holders broke instantly. Will ...              0   
5  I use this as a display stand for our pick col...              1   

   is_verified_purchase                                      product_title  \
0                  True  VoiceBooster Voice Amplifier 20watts Black MR2...   
1                  True  8-pack Potentiometer Knob

In [27]:
na_count = df_clean['review_title'].isna().sum()
na_count_text = df_clean['review_text'].isna().sum()
total_rows = len(df_clean)
print(f"Anzahl fehlender 'full_review'-Einträge: {na_count} von {total_rows} ({(na_count/total_rows)*100:.2f}%)")
print(f"Anzahl fehlender 'full_review'-Einträge: {na_count_text} von {total_rows} ({(na_count_text/total_rows)*100:.2f}%)")
df_clean = df_clean.dropna(subset=['review_title'])
df_clean = df_clean.dropna(subset=['review_text'])

Anzahl fehlender 'full_review'-Einträge: 19 von 129669 (0.01%)
Anzahl fehlender 'full_review'-Einträge: 34 von 129669 (0.03%)


In [45]:
df_clean['review_title_lemmatized'] = df_clean['review_title'].apply(lemmatize_text)
df_clean['review_text_lemmatized'] = df_clean['review_text'].apply(lemmatize_text)
df_clean['full_review'] = df_clean['review_title_lemmatized'] + ' ' + df_clean['review_text_lemmatized']

In [46]:
df_clean['word_count'] = df_clean['full_review'].apply(lambda x: len(x.split()))
df_clean['char_count'] = df_clean['full_review'].apply(lambda x: len(x))
df_clean['avg_word_length'] = df_clean['char_count'] / df_clean['word_count']

In [47]:
display(df_clean)

,review_rating,review_title,review_text,helpful_votes,is_verified_purchase,product_title,product_average_rating,product_rating_count,full_review,word_count,char_count,avg_word_length,review_title_lemmatized,review_text_lemmatized
0,5,Most Excellent,Works great and sounds out loud!,0,True,VoiceBooster Voice Amplifier 20watts Black MR2...,4.6,107,excellent work great sound loud,5,31,6.200000,excellent,work great sound loud
1,3,Three Stars,"The knobs seem like okay quality, but they don...",5,True,8-pack Potentiometer Knobs: Black Ribbed with ...,3.8,19,three star knobs seem like okay quality fit am...,10,53,5.300000,three star,knobs seem like okay quality fit ampeg svt
2,5,Very durable and produces a great sound,Very durable and produce great sound. I person...,0,True,ChromaCast 96 Pick Sampler - Includes Pearl Ce...,4.7,9760,durable produce great sound durable produce gr...,20,138,6.900000,durable produce great sound,durable produce great sound personally love ch...
4,3,Meh,One of the pick holders broke instantly. Will ...,0,True,20 Pieces Metal Guitar Picks Plectrums Stainle...,4.4,1243,meh one pick holders break instantly need supe...,14,81,5.785714,meh,one pick holders break instantly need superglu...
5,4,Displays our Pick Collection,I use this as a display stand for our pick col...,1,True,D'Addario Accessories Mic Stand Pick Holder,4.6,387,display pick collection use display stand pick...,8,58,7.250000,display pick collection,use display stand pick collections
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149879,5,Perfect,Granddaughter loves it. She's in college as a ...,0,True,Manhasset Model #48 Sheet Music Stand,4.8,4284,perfect granddaughter love college music ed major,7,49,7.000000,perfect,granddaughter love college music ed major
149880,5,Simply perfect,Bought this stand for my son for his birthday ...,1,True,Ultimate Support JamStands Trumpet Stand Black,3.7,3,simply perfect buy stand son birthday could ha...,23,145,6.304348,simply perfect,buy stand son birthday could happier say stand...
149881,5,Refunded quickly and efficiently.,It wasn't what I wanted. Refiunded quickly an...,0,True,"Yamaha MG10 10-Input Stereo Mixer , Black",4.6,4683,refund quickly efficiently want refiunded quic...,10,81,8.100000,refund quickly efficiently,want refiunded quickly efficiently thank selle...
149882,1,Not the handwired version,"Not a handwired deep blue delay, just the mass...",0,True,Mad Professor MAD-HW-DBD Guitar Delay Effects ...,3.9,3,handwired version handwired deep blue delay ma...,9,60,6.666667,handwired version,handwired deep blue delay mass produce one


In [48]:
from nltk.corpus import opinion_lexicon

# Laden der positiven und negativen Wörter
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())

In [49]:
def count_sentiment_words(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    pos_count = sum(1 for word in tokens if word in positive_words)
    neg_count = sum(1 for word in tokens if word in negative_words)
    return pos_count, neg_count

df_clean[['positive_word_count', 'negative_word_count']] = df_clean['full_review'].apply(
    lambda x: pd.Series(count_sentiment_words(x))
)

In [50]:
together_count = df_clean['products_bought_together'].isna().sum()
print(f"Anzahl fehlender 'products_bought_together'-Einträge: {together_count} von {total_rows} ({(together_count/total_rows)*100:.2f}%)")
df_clean = df_clean.drop(columns=['products_bought_together'])

KeyError: 'products_bought_together'

In [53]:
df_clean['log_helpful_votes'] = np.log1p(df_clean['helpful_votes'])

In [56]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_clean[['product_average_rating', 'product_rating_count', 'log_helpful_votes']] = scaler.fit_transform(
    df_clean[['product_average_rating', 'product_rating_count', 'log_helpful_votes']]
)

In [57]:
# Erstellung des TF-IDF-Vektors
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_features = tfidf.fit_transform(df_clean['full_review'])

In [64]:
import numpy as np

# Liste der numerischen Features
numerical_features = ['word_count', 'char_count', 'avg_word_length', 
                      'positive_word_count', 'negative_word_count',
                      'product_average_rating', 'product_rating_count', 'log_helpful_votes']

# Überprüfen auf infinity und NaN
for feature in numerical_features:
    num_inf = np.isinf(df_clean[feature]).sum()
    num_nan = df_clean[feature].isna().sum()
    print(f"Feature '{feature}': {num_inf} Inf-Werte, {num_nan} NaN-Werte")

# Ersetzen von Inf-Werten durch NaN
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

# Finden und Entfernen von Zeilen, wo word_count == 0
zero_word_count = (df_clean['word_count'] == 0).sum()
print(f"Anzahl der Zeilen mit word_count == 0: {zero_word_count}")

df_clean = df_clean[df_clean['word_count'] != 0]

# Überprüfen der NaN-Werte nach den bisherigen Schritten
for feature in numerical_features:
    num_nan = df_clean[feature].isna().sum()
    print(f"Feature '{feature}': {num_nan} NaN-Werte nach Behandlung von Inf-Werten und word_count")

# Imputieren der NaN-Werte, z.B. mit dem Median
df_clean[numerical_features] = df_clean[numerical_features].fillna(df_clean[numerical_features].median())

# Finaler Check
for feature in numerical_features:
    if np.isinf(df_clean[feature]).any():
        print(f"Feature '{feature}' enthält noch Infinity-Werte.")
    if df_clean[feature].isna().any():
        print(f"Feature '{feature}' enthält noch NaN-Werte.")

from sklearn.preprocessing import StandardScaler

# Initialisieren des Scalers
scaler = StandardScaler()

# Skalieren der numerischen Features
df_clean[numerical_features] = scaler.fit_transform(df_clean[numerical_features])


Feature 'word_count': 0 Inf-Werte, 0 NaN-Werte
Feature 'char_count': 0 Inf-Werte, 0 NaN-Werte
Feature 'avg_word_length': 107 Inf-Werte, 0 NaN-Werte
Feature 'positive_word_count': 0 Inf-Werte, 0 NaN-Werte
Feature 'negative_word_count': 0 Inf-Werte, 0 NaN-Werte
Feature 'product_average_rating': 0 Inf-Werte, 0 NaN-Werte
Feature 'product_rating_count': 0 Inf-Werte, 0 NaN-Werte
Feature 'log_helpful_votes': 0 Inf-Werte, 0 NaN-Werte
Anzahl der Zeilen mit word_count == 0: 107
Feature 'word_count': 0 NaN-Werte nach Behandlung von Inf-Werten und word_count
Feature 'char_count': 0 NaN-Werte nach Behandlung von Inf-Werten und word_count
Feature 'avg_word_length': 0 NaN-Werte nach Behandlung von Inf-Werten und word_count
Feature 'positive_word_count': 0 NaN-Werte nach Behandlung von Inf-Werten und word_count
Feature 'negative_word_count': 0 NaN-Werte nach Behandlung von Inf-Werten und word_count
Feature 'product_average_rating': 0 NaN-Werte nach Behandlung von Inf-Werten und word_count
Feature 'pro

/var/folders/hz/2fntg1q133z6s0ff1j1vhkgm0000gn/T/ipykernel_67555/3934875794.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[numerical_features] = df_clean[numerical_features].fillna(df_clean[numerical_features].median())
/var/folders/hz/2fntg1q133z6s0ff1j1vhkgm0000gn/T/ipykernel_67555/3934875794.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[numerical_features] = scaler.fit_transform(df_clean[numerical_features])


In [65]:
display(df_clean.head())
print(df_clean.dtypes)

,review_rating,review_title,review_text,helpful_votes,is_verified_purchase,product_title,product_average_rating,product_rating_count,full_review,word_count,char_count,avg_word_length,review_title_lemmatized,review_text_lemmatized,positive_word_count,negative_word_count,log_helpful_votes
0,5,Most Excellent,Works great and sounds out loud!,0,1,VoiceBooster Voice Amplifier 20watts Black MR2...,0.536980,-0.364904,excellent work great sound loud,-0.558163,-0.552407,0.119485,excellent,work great sound loud,-0.221371,-0.068186,-0.515597
1,3,Three Stars,"The knobs seem like okay quality, but they don...",5,1,8-pack Potentiometer Knobs: Black Ribbed with ...,-1.413961,-0.374859,three star knobs seem like okay quality fit am...,-0.418476,-0.454618,-1.125748,three star,knobs seem like okay quality fit ampeg svt,-0.732267,-0.524589,2.244666
2,5,Very durable and produces a great sound,Very durable and produce great sound. I person...,0,1,ChromaCast 96 Pick Sampler - Includes Pearl Ce...,0.780847,0.727112,durable produce great sound durable produce gr...,-0.139103,-0.076799,1.088001,durable produce great sound,durable produce great sound personally love ch...,0.800421,-0.524589,-0.515597
4,3,Meh,One of the pick holders broke instantly. Will ...,0,1,20 Pieces Metal Guitar Picks Plectrums Stainle...,0.049245,-0.236391,meh one pick holders break instantly need supe...,-0.306727,-0.330160,-0.453717,meh,one pick holders break instantly need superglu...,-0.732267,-0.068186,-0.515597
5,4,Displays our Pick Collection,I use this as a display stand for our pick col...,1,1,D'Addario Accessories Mic Stand Pick Holder,0.536980,-0.333228,display pick collection use display stand pick...,-0.474351,-0.432394,1.572258,display pick collection,use display stand pick collections,-0.987715,-0.524589,0.552219


review_rating                int64
review_title                object
review_text                 object
helpful_votes                int64
is_verified_purchase         int64
product_title               object
product_average_rating     float64
product_rating_count       float64
full_review                 object
word_count                 float64
char_count                 float64
avg_word_length            float64
review_title_lemmatized     object
review_text_lemmatized      object
positive_word_count        float64
negative_word_count        float64
log_helpful_votes          float64
dtype: object


In [61]:
# Konvertiere boolsche Werte in numerische Werte
df_clean['is_verified_purchase'] = df_clean['is_verified_purchase'].astype(int)